In [158]:
import numpy as np

X_world = np.array([10,10,10000,1]) #a really big depth point

#assuming world frame is same as camera 1
X_camera1 = np.array([10,10,10000,1])
X_camera1 = np.array([0.001,0.001,1,0.0001])

def estimate_E(xy1, xy2):
    n = xy1.shape[1]
    A = np.zeros((n, 9))
    
    for i in range(n):
        x1 = xy1[0,i]
        y1 = xy1[1,i]
        x2 = xy2[0,i]
        y2 = xy2[1,i]
        
        A[i,:] = np.array([x2*x1, x2*y1, x2, y2*x1, y2*y1, y2, x1, y1, 1])
    
    #get V.T:
    _,_,VT = np.linalg.svd(A)
    V = VT.T
    e = V[:,-1]

    E = np.reshape(e,(3,3))    
    return E



def SE3(R,t):
    T = np.eye(4)
    T[:3,:3] = R
    T[:3,3] = t
    return T

def decompose_E(E):
    """
    Computes the four possible decompositions of E into a relative
    pose, as described in Szeliski 7.2.

    Returns a list of 4x4 transformation matrices.
    """
    U,_,VT = np.linalg.svd(E)
    R90 = np.array([[0, -1, 0], [+1, 0, 0], [0, 0, 1]])
    R1 = U @ R90 @ VT
    R2 = U @ R90.T @ VT
    if np.linalg.det(R1) < 0: R1 = -R1
    if np.linalg.det(R2) < 0: R2 = -R2
    t1, t2 = U[:,2], -U[:,2]
    return [SE3(R1,t1), SE3(R1,t2), SE3(R2, t1), SE3(R2, t2)]

def triangulate_many(xy1, xy2, P1, P2):
    """
    Arguments
        xy: Calibrated image coordinates in image 1 and 2
            [shape 3 x n]
        P:  Projection matrix for image 1 and 2
            [shape 3 x 4]
    Returns
        X:  Dehomogenized 3D points in world frame
            [shape 4 x n]
    """
    n = xy1.shape[1]
    X = np.zeros((4,n)) # Placeholder, replace with your implementation
    
    #SVD of A to get 3D world coordinates
    for i in range(n):
        x1 = xy1[0,i]
        y1 = xy1[1,i]
        x2 = xy2[0,i]
        y2 = xy2[1,i]
        
        
        A = np.array([x1*P1[2,:] - P1[0,:],
                      y1*P1[2,:] - P1[1,:],
                      x2*P2[2,:] - P2[0,:],
                      y2*P2[2,:] - P2[1,:]])
        U, R, V = np.linalg.svd(A)
        VT = V.T
        X[:,i] = VT[:,-1]

    UX = X
    # NORMALIZING
    X = X/X[-1]
    
    return X,UX


In [270]:
#camera coords to film coords:
K = np.loadtxt('../data/K.txt')
xy1_tilde = np.array([[1],[42],[10000000000000]])
xy2_tilde = np.array([[22],[13],[10000000000000]])

xy1 = xy1_tilde[:2,:]/xy1_tilde[2,:]
xy2 = xy2_tilde[:2,:]/xy2_tilde[2,:]


In [271]:
E = estimate_E(xy1, xy2)

In [272]:
P1 = np.hstack([np.eye(3),np.zeros((3,1))])

#Better way of choosing P2 based on total positive z values of all X_camera2 correspondences
def choose_best_E_for_P2(decomposed_E):
    max_positive_z  = 0
    for E_matrix in decomposed_E:
        P2 = E_matrix[:3,:]
        X,_ = triangulate_many(xy1, xy2, P1, P2)
        X_camera2 = P2@X
        
        possible_max = np.sum((X_camera2[2] >= 0))
        if max_positive_z < possible_max:
            max_positive_z = possible_max
            best_P2_matrix = E_matrix
    return best_P2_matrix

P2 = choose_best_E_for_P2(decompose_E(E))
X,UX = triangulate_many(xy1, xy2, P1, P2)

In [273]:
X
print(UX)

[[-7.07106781e-14]
 [-1.34800117e-12]
 [-7.07106781e-01]
 [-7.07106781e-01]]
